In [30]:
%pip install --upgrade --quiet  wikipedia langchain_openai

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dspy-ai 2.4.5 requires pydantic==2.5.0, but you have pydantic 2.6.4 which is incompatible.
instructor 1.2.1 requires pydantic==2.7.0, but you have pydantic 2.6.4 which is incompatible.
instructor 1.2.1 requires pydantic-core<3.0.0,>=2.18.0, but you have pydantic-core 2.16.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [57]:
from indexify import IndexifyClient, ExtractionGraph
client = IndexifyClient()

In [58]:
extraction_graph_spec = """
name: 'wikipedia-knowledgebase'
extraction_policies:
   - extractor: 'tensorlake/chunk-extractor'
     name: 'chunker'
     input_params:
        chunk_size: 1000
        overlap: 100
   - extractor: 'tensorlake/minilm-l6'
     name: 'wikiembedding'
     content_source: 'chunker'
"""

extraction_graph = ExtractionGraph.from_yaml(extraction_graph_spec)
client.create_extraction_graph(extraction_graph)                                            

In [37]:
from langchain_community.document_loaders import WikipediaLoader

In [59]:
docs = WikipediaLoader(query="Kevin Durant", load_max_docs=1).load()
for doc in docs:
    client.add_documents("wikipedia-knowledgebase", doc.page_content)                 

In [60]:
client.indexes()

[{'name': 'wikipedia-extractor.wikiembedding.embedding',
  'embedding_schema': {'dim': 384, 'distance': 'cosine'}}]

In [31]:
from indexify_langchain import IndexifyRetriever

In [61]:
params = {"name": "wikipedia-knowledgebase.wikiembedding.embedding", "top_k": 50}
retriever = IndexifyRetriever(client=client, params=params)                               
 

In [62]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

In [63]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(openai_api_key="")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


In [64]:
chain.invoke("When and where did Kevin Durant win his championships?") 

'Kevin Wayne Durant won his championships in the NBA while playing for the Golden State Warriors in 2017 and 2018.'